In [2]:
# get openai api key from openai_config.json file
import json
with open('openai_config.json') as f:
    config =json.load(f)
    NCP_CLOVASTUDIO_API_KEY = config['NCP_CLOVASTUDIO_API_KEY']
    NCP_APIGW_API_KEY = config['NCP_APIGW_API_KEY']
    NCP_CLOVASTUDIO_APP_ID = config['NCP_CLOVASTUDIO_APP_ID']

In [3]:
%env NCP_CLOVASTUDIO_API_KEY = {NCP_CLOVASTUDIO_API_KEY}
%env NCP_APIGW_API_KEY = {NCP_APIGW_API_KEY}
%env NCP_CLOVASTUDIO_APP_ID = {NCP_CLOVASTUDIO_APP_ID}

env: NCP_CLOVASTUDIO_API_KEY=NTA0MjU2MWZlZTcxNDJiY2ns+hsihDXAOl/Qw9LFrJ6TPXSiWUtFwboMBVzZKgTJ
env: NCP_APIGW_API_KEY=22GlkTNNdZ3XJ4RELCauPBknjMc8XAXhbIzT1qFJ
env: NCP_CLOVASTUDIO_APP_ID=c7ddde1ea597417797d63a039eca05d7


In [4]:
from langchain_community.chat_models import ChatClovaX
chat = ChatClovaX(
    model="HCX-003",
    max_tokens=1000,
    temperature=0.5,)

In [5]:
novel_prompt = """너는 단문 소설을 쓰는 작가야. 주어진 단어를 포함하는 세 문단짜리 단문 소설을 작문해야 해.
아래 단어를 모두 포함하도록 단문 소설을 써 줘.
{keywords}
제목 없이 소설 내용만 써 줘. 주어진 단어를 활용하는 자연스러운 전개로 써 줘.
전지적 작가 시점으로 써 줘.
인물의 대사 한 개 이상을 포함해.
소설은 세 문단으로 이루어져야 해. 세 문단 이상 쓰지 마.
"""
science_prompt = """너는 과학적 사실을 설명하는 글을 쓰는 과학 저널리스트야. 주어진 과학 질문에 대한 답변을 세 문단으로 작성해야 해.
아래 과학 질문에 대한 답변을 써 줘.
{question}
제목이나 요약 없이 답변 내용만 써 줘. 질문과 관련없는 내용은 쓰지 마.
-다, 와 같은 종결어미를 사용하는 해라체로 작성해.
답변은 세 문단으로 이루어져야 해.
"""

In [7]:
# import csv

# results = []
# with open('hw4_data/sample_novel.csv', 'r', newline='') as csvfile:
#     reader = csv.reader(csvfile)
#     for row in reader:
#         keywords = row
#         novel_messages = [
#             (
#                 "system",
#                 "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.",
#             ),
#             ("human", novel_prompt.format(keywords=row)),
#         ]
#         ai_msg = chat.invoke(novel_messages)
#         results.append((keywords, ai_msg.content))

In [ ]:
# with open('hw4_results/sample_novel_results.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     for res in results:
#         writer.writerow(res)

In [8]:
import csv

results = []
with open('hw4_data/science_questions.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        keywords = row
        novel_messages = [
            (
                "system",
                "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.",
            ),
            ("human", science_prompt.format(question=row)),
        ]
        ai_msg = chat.invoke(novel_messages)
        results.append((keywords, ai_msg.content))

In [12]:
results[0]

(['\ufeff정제 탄수화물과 비정제 탄수화물은 사람의 소화 과정에 어떤 차이점을 보이는가?'],
 '정제 탄수화물과 비정제 탄수화물은 사람의 소화 과정에서 서로 다른 특징을 보인다.\n\n먼저, 정제 탄수화물은 인공적으로 가공되어 만들어진 탄수화물이다. 일반적으로 녹말, 설탕, 밀가루 등이 이에 해당한다. 정제 탄수화물은 소화 흡수가 빠르고 혈당을 빠르게 상승시키기 때문에, 인슐린 분비를 촉진하여 비만, 당뇨병 등의 질환을 유발할 수 있다. 또한, 체내에서 빠르게 분해되기 때문에 에너지 지속 시간이 짧다는 단점도 있다.\n\n반면, 비정제 탄수화물은 자연 상태 그대로의 탄수화물로서, 과일, 채소, 견과류, 통곡물 등이 이에 해당한다. 비정제 탄수화물은 소화 흡수가 천천히 일어나며, 혈당을 완만하게 상승시킨다. 이로 인해 인슐린 분비가 억제되어 비만, 당뇨병 등의 질환 예방에 도움이 된다. 또, 체내에서 느리게 분해되기 때문에 에너지 지속 시간이 길다는 장점도 있다.\n\n따라서, 건강을 위해서는 비정제 탄수화물을 섭취하는 것이 좋다. 하지만, 개인의 체질과 상황에 따라 적절한 양과 종류를 선택하는 것이 중요하다. 예를 들어, 당뇨병 환자나 다이어트 중인 사람은 과도한 비정제 탄수화물 섭취를 자제해야 한다.')

In [11]:
import pandas as pd


# Convert the list into a DataFrame
df = pd.DataFrame(results, columns=["question", "answer"])  # You can rename "Names" to any column header

# Write to CSV file
df.to_csv("hw4_results/clova_science_results.csv", index=False)

print("File written successfully!")


File written successfully!
